In [1]:
# !pip install --upgrade tensorflow --quiet
# !pip install keras_tuner --quiet
# !pip install tensorflow-io --quiet
# # Google colab modules
# from google.colab import drive
import sys, importlib

# # Mount drive
# drive.mount('/content/gdrive', force_remount=True)
ROOT_PATH = './'
# sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()
# core._init_models()
# core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

In [2]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


In [3]:
model_base_id = '327'
version = '5'
model_no = model_base_id
constituent_limits = 0.35
id_constituent = 1
min_vid_constituents = 0.2
epochs = 5
batch_size = 1024
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 12579
Total constituents: 422
Epoch 1-1/5
2503/2503 [==============================] - 2003s 797ms/step - loss: 0.8323 - accuracy: 0.5207 - val_loss: 0.7400 - val_accuracy: 0.5264
Epoch 1-2/5
2486/2486 [==============================] - 1998s 803ms/step - loss: 0.7126 - accuracy: 0.5391 - val_loss: 0.7036 - val_accuracy: 0.5431
Epoch 1-3/5
2489/2489 [==============================] - 1999s 803ms/step - loss: 0.6924 - accuracy: 0.5543 - val_loss: 0.6877 - val_accuracy: 0.5581
Epoch 1-4/5
2480/2480 [==============================] - 1998s 805ms/step - loss: 0.6820 - accuracy: 0.5681 - val_loss: 0.6788 - val_accuracy: 0.5718
Epoch 1-5/5
2491/2491 [==============================] - 2004s 804ms/step - loss: 0.6749 - accuracy: 0.5792 - val_loss: 0.6725 - val_accuracy: 0.5839
Epoch 1-6/5
2513/2513 [==============================] - 2038s 811ms/step - loss: 0.6710 - accuracy: 0.5846 - val_loss: 0.6699 - val_accuracy: 0.5883
Epoch 1-7/5
2542/2542 [===========================

ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  Failed to allocate memory for the batch of component 0
	 [[node IteratorGetNext
 (defined at C:\Users\Jonathan Raditya\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py:866)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[model/lstm/Shape/_4]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  Failed to allocate memory for the batch of component 0
	 [[node IteratorGetNext
 (defined at C:\Users\Jonathan Raditya\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py:866)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, 